## Import Statements and Read in Files

In [ ]:
# Import Statements
import pandas as pd
import numpy as np

import geopy
from geopy.distance import geodesic

In [174]:
df1 = pd.read_csv('C:/Users/grace/OneDrive/Desktop/GMU/DAEN690/std_format_uas_loc.csv')

In [175]:
df1

,REMARKS,UAS_LOC,IDENT,NAME,LATITUDE,LONGITUDE,ICAO_ID
0,Aircraft observed alarge orange UAS with flash...,['3 NM NNW of CLE'],CLE,Cleveland-Hopkins Intl,41.409407,-81.854691,KCLE
1,"Aircraft observed a UAS while at 5,000 feet 11...",['11 NM SSE of SLC'],SLC,Salt Lake City Intl,40.788388,-111.977773,KSLC
2,"Aircraft observed a UAS while at 5,000 feet 11...",['11 NM SSE of SLC'],SLC,Salt Lake City Intl,40.788388,-111.977773,KSLC
3,Aircraft observed a multi-colored UAS off the ...,['7NM E of SAN'],SAN,San Diego Intl,32.733556,-117.189667,KSAN
4,"Aircraft observed a triangular shaped, grey an...",['23NM SW of ORL'],ORL,Exec,28.545462,-81.332930,KORL
...,...,...,...,...,...,...,...
3028,Aircraft reported a quad-copter UAS at 12 O'cl...,['10 NM SE of ADS'],ADS,Addison,32.968556,-96.836444,KADS
3029,Aircraft reported a UAS 3 NM N of MMU while E ...,['3 NM N of MMU'],MMU,Morristown Muni,40.799338,-74.414889,KMMU
3030,Aircraft reported a UAS sensor hit while N bou...,['21 NM NW of BXK'],BXK,Buckeye Muni,33.420417,-112.686181,KBXK
3031,From MOR: Aircraft reported a NMAC with a flat...,['5 NM ENE of DRK'],DRK,NaN,NaN,NaN,NaN


#### Drop all NULL values from DataFrame 

2,818 UAS sightings following a Standard Format remain

In [176]:
df1 = df1.dropna().reset_index() # DROP ALL NULL VALUES FOR THE TIME BEING (LEAVES 2818 RECORDS)
df1

,index,REMARKS,UAS_LOC,IDENT,NAME,LATITUDE,LONGITUDE,ICAO_ID
0,0,Aircraft observed alarge orange UAS with flash...,['3 NM NNW of CLE'],CLE,Cleveland-Hopkins Intl,41.409407,-81.854691,KCLE
1,1,"Aircraft observed a UAS while at 5,000 feet 11...",['11 NM SSE of SLC'],SLC,Salt Lake City Intl,40.788388,-111.977773,KSLC
2,2,"Aircraft observed a UAS while at 5,000 feet 11...",['11 NM SSE of SLC'],SLC,Salt Lake City Intl,40.788388,-111.977773,KSLC
3,3,Aircraft observed a multi-colored UAS off the ...,['7NM E of SAN'],SAN,San Diego Intl,32.733556,-117.189667,KSAN
4,4,"Aircraft observed a triangular shaped, grey an...",['23NM SW of ORL'],ORL,Exec,28.545462,-81.332930,KORL
...,...,...,...,...,...,...,...,...
2813,3027,Aircraft reported a medium sized quad-copter U...,['3 NM SW of SAT'],SAT,San Antonio Intl,29.533958,-98.469057,KSAT
2814,3028,Aircraft reported a quad-copter UAS at 12 O'cl...,['10 NM SE of ADS'],ADS,Addison,32.968556,-96.836444,KADS
2815,3029,Aircraft reported a UAS 3 NM N of MMU while E ...,['3 NM N of MMU'],MMU,Morristown Muni,40.799338,-74.414889,KMMU
2816,3030,Aircraft reported a UAS sensor hit while N bou...,['21 NM NW of BXK'],BXK,Buckeye Muni,33.420417,-112.686181,KBXK


## Create Dictionary of Bearings and Respective Degrees

In [32]:
bearing_deg = {
    'N' : 0,
    'NNE' : 23,
    'NE' : 45,
    'ENE' : 68,
    'E' : 90,
    'ESE' : 113,
    'SE' : 135,
    'SSE' : 158,
    'S' : 180,
    'SSW' : 203,
    'SW' : 225,
    'WSW' : 248,
    'W' : 270,
    'WNW' : 293,
    'NW' : 315,
    'NNW' : 338
}

In [117]:
# bearing_deg.values()
# bearing_deg.keys()
# bearing_deg.items()

#### Break Up UAS_LOC column into the Distances and Bearing Abbreviations 

In [177]:
# Extract Distances and Bearings from UAS_LOC that were pulled from Remarks
# with a Standard Format in SkyWatch
uas_loc = df1['UAS_LOC']
distances = []
bearings = []

for i in range(len(uas_loc)):
    split = uas_loc[i].split(' ')[0:3]
    
    if 'NM' in split[0]: #if there is no space between the distance and NM (i.e. 3NM)
        distance_nm = split[0][0:len(split[0]) - 2]
        bearing_abrev = split[1].replace('of', '')
        
    else: 
        distance_nm = split[0]
        bearing_abrev = split[2].replace('of', '')
    
    distances.append(float(distance_nm[2:]))
    bearings.append(bearing_abrev)

In [216]:
# Create DataFrame to store UAS Location Information
new_df = pd.DataFrame(df1['REMARKS'])
new_df['UAS_LOC'] = df1['UAS_LOC']
new_df['IDENT'] = df1['IDENT']
new_df['Distance'] = distances
new_df['Bearing'] = bearings

new_df

,REMARKS,UAS_LOC,IDENT,Distance,Bearing
0,Aircraft observed alarge orange UAS with flash...,['3 NM NNW of CLE'],CLE,3.0,NNW
1,"Aircraft observed a UAS while at 5,000 feet 11...",['11 NM SSE of SLC'],SLC,11.0,SSE
2,"Aircraft observed a UAS while at 5,000 feet 11...",['11 NM SSE of SLC'],SLC,11.0,SSE
3,Aircraft observed a multi-colored UAS off the ...,['7NM E of SAN'],SAN,7.0,E
4,"Aircraft observed a triangular shaped, grey an...",['23NM SW of ORL'],ORL,23.0,SW
...,...,...,...,...,...
2813,Aircraft reported a medium sized quad-copter U...,['3 NM SW of SAT'],SAT,3.0,SW
2814,Aircraft reported a quad-copter UAS at 12 O'cl...,['10 NM SE of ADS'],ADS,10.0,SE
2815,Aircraft reported a UAS 3 NM N of MMU while E ...,['3 NM N of MMU'],MMU,3.0,N
2816,Aircraft reported a UAS sensor hit while N bou...,['21 NM NW of BXK'],BXK,21.0,NW


#### Convert all Distances from NM to Kilometers and Bearing Abbreviations into Degrees

In [217]:
dist_kilo = []

for i in range(len(new_df)):
    distanceKilo = new_df['Distance'][i] * 1.852 # converting NM to kilometers
    dist_kilo.append(distanceKilo)

bearDegree = pd.DataFrame(new_df['Bearing'])
bearDegree = bearDegree.replace({'Bearing': bearing_deg})

new_df['Distance_Kilometers'] = dist_kilo
new_df['Bearing_Degrees'] = bearDegree
new_df['Airport_Latitude'] = df1['LATITUDE']
new_df['Airport_Longitude'] = df1['LONGITUDE']

In [218]:
new_df

,REMARKS,UAS_LOC,IDENT,Distance,Bearing,Distance_Kilometers,Bearing_Degrees,Airport_Latitude,Airport_Longitude
0,Aircraft observed alarge orange UAS with flash...,['3 NM NNW of CLE'],CLE,3.0,NNW,5.556,338,41.409407,-81.854691
1,"Aircraft observed a UAS while at 5,000 feet 11...",['11 NM SSE of SLC'],SLC,11.0,SSE,20.372,158,40.788388,-111.977773
2,"Aircraft observed a UAS while at 5,000 feet 11...",['11 NM SSE of SLC'],SLC,11.0,SSE,20.372,158,40.788388,-111.977773
3,Aircraft observed a multi-colored UAS off the ...,['7NM E of SAN'],SAN,7.0,E,12.964,90,32.733556,-117.189667
4,"Aircraft observed a triangular shaped, grey an...",['23NM SW of ORL'],ORL,23.0,SW,42.596,225,28.545462,-81.332930
...,...,...,...,...,...,...,...,...,...
2813,Aircraft reported a medium sized quad-copter U...,['3 NM SW of SAT'],SAT,3.0,SW,5.556,225,29.533958,-98.469057
2814,Aircraft reported a quad-copter UAS at 12 O'cl...,['10 NM SE of ADS'],ADS,10.0,SE,18.520,135,32.968556,-96.836444
2815,Aircraft reported a UAS 3 NM N of MMU while E ...,['3 NM N of MMU'],MMU,3.0,N,5.556,0,40.799338,-74.414889
2816,Aircraft reported a UAS sensor hit while N bou...,['21 NM NW of BXK'],BXK,21.0,NW,38.892,315,33.420417,-112.686181


#### Remove records from DataFrame with no Bearing Information Extracted

2,812 UAS sighting records remain

In [219]:
new_df['Bearing'].replace('', np.nan, inplace = True)
new_df.dropna(subset = ['Bearing'], inplace = True) #LEAVES 2812 ROWS

new_df = new_df.reset_index()

#### Calculate Lat/Long Coordinates for Sighted UAS using geopy

In [220]:
uas_lat = []
uas_long = []


for i in range(len(new_df)):
    lat_airport = pd.to_numeric(new_df['Airport_Latitude'][i])
    long_airport = pd.to_numeric(new_df['Airport_Longitude'][i])
    b = pd.to_numeric(new_df['Bearing_Degrees'][i])
    d = pd.to_numeric(new_df['Distance_Kilometers'][i])
    
    origin = geopy.Point(lat_airport, long_airport)
    destination = geodesic(kilometers=d).destination(origin,b)
    
    lat2, lon2, = destination.latitude, destination.longitude
    
    uas_lat.append(lat2)
    uas_long.append(lon2)

In [221]:
# Append UAS Lat/Long information to DataFrame
new_df['UAS_Latitude'] = uas_lat
new_df['UAS_Longitude'] = uas_long

In [222]:
new_df

,index,REMARKS,UAS_LOC,IDENT,Distance,Bearing,Distance_Kilometers,Bearing_Degrees,Airport_Latitude,Airport_Longitude,UAS_Latitude,UAS_Longitude
0,0,Aircraft observed alarge orange UAS with flash...,['3 NM NNW of CLE'],CLE,3.0,NNW,5.556,338,41.409407,-81.854691,41.455788,-81.879601
1,1,"Aircraft observed a UAS while at 5,000 feet 11...",['11 NM SSE of SLC'],SLC,11.0,SSE,20.372,158,40.788388,-111.977773,40.618259,-111.887586
2,2,"Aircraft observed a UAS while at 5,000 feet 11...",['11 NM SSE of SLC'],SLC,11.0,SSE,20.372,158,40.788388,-111.977773,40.618259,-111.887586
3,3,Aircraft observed a multi-colored UAS off the ...,['7NM E of SAN'],SAN,7.0,E,12.964,90,32.733556,-117.189667,32.733479,-117.051359
4,4,"Aircraft observed a triangular shaped, grey an...",['23NM SW of ORL'],ORL,23.0,SW,42.596,225,28.545462,-81.332930,28.273338,-81.639923
...,...,...,...,...,...,...,...,...,...,...,...,...
2807,2813,Aircraft reported a medium sized quad-copter U...,['3 NM SW of SAT'],SAT,3.0,SW,5.556,225,29.533958,-98.469057,29.498509,-98.509572
2808,2814,Aircraft reported a quad-copter UAS at 12 O'cl...,['10 NM SE of ADS'],ADS,10.0,SE,18.520,135,32.968556,-96.836444,32.850395,-96.696550
2809,2815,Aircraft reported a UAS 3 NM N of MMU while E ...,['3 NM N of MMU'],MMU,3.0,N,5.556,0,40.799338,-74.414889,40.849370,-74.414889
2810,2816,Aircraft reported a UAS sensor hit while N bou...,['21 NM NW of BXK'],BXK,21.0,NW,38.892,315,33.420417,-112.686181,33.668009,-112.982708


In [223]:
# Export to .csv file
new_df.to_csv('uas_latlong_standard_2812.csv', index = False)

## Visualizations/Plotting UAS Sightings

In [57]:
import plotly.graph_objects as go

import pandas as pd

In [224]:
one = np.unique(new_df['IDENT'].values)  #set the array of unique IDENTS in your column, new_df['IDENT']
d = dict(zip(one, np.arange(len(one)))) # a dict that associates a numerical value to each IDENT

fig = go.Figure(data=go.Scattergeo(
        lon = new_df['UAS_Longitude'],
        lat = new_df['UAS_Latitude'],
        text = new_df['IDENT'],
        mode = 'markers',
        marker_color = [d[s] for s in new_df['IDENT']]
        
        ))

fig.update_layout(
        title = 'UAS Sightings Reported through SkyWatch with Standard Format<br>(Hover for Nearest Airport Name)',
        geo_scope='usa',
    )


fig.show()